In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-spanner-python/blob/main/samples/langchain_quick_start.ipynb)

---
# **Introduction**

In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [Spanner](https://cloud.google.com/spanner) and [LangChain](https://www.langchain.com/). We will be creating an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What you'll learn

* How to deploy a Spanner instance
* How to use Spanner as a VectorStore
* How to use Spanner as a DocumentLoader
* How to use Spanner for ChatHistory storage

## What you'll need
* A Google Cloud Account and Google Cloud Project
* A web browser such as [Chrome](https://www.google.com/chrome/)

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for our application
2. Set up authentication for our project
3. Set up a Spanner Instance
4. Import the data used by our application

## Install dependencies
First you will need to install the dependencies needed to run this demo app.

In [ ]:
%pip install langchain-google-spanner

%pip install langchain langchain-google-vertexai

## Authenticate to Google Cloud within Colab
In order to access your Google Cloud Project from this notebook, you will need to Authenticate as an IAM user.

In [ ]:
from google.colab import auth

auth.authenticate_user()

## Connect Your Google Cloud Project
Time to connect your Google Cloud Project to this notebook so that you can leverage Google Cloud from within Colab.

In [ ]:
# @markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "your_project_id"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}
%env GOOGLE_CLOUD_PROJECT={project_id}

## Configure Your Google Cloud Project
Configure the following in your Google Cloud Project.

1. IAM principal (user, service account, etc.) with the
[Spanner Database User][client-role] role.

> The user logged into this notebook will be used as the IAM principal and will be granted the Spanner Database Use  role.

[client-role]: https://cloud.google.com/spanner/docs/iam#sessions

In [ ]:
# grant Cloud Spanner databaseUser role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/spanner.databaseUser" \
  --condition=None

2. Enable the APIs for Spanner and Vertex AI within your project.

In [ ]:
# enable GCP services
!gcloud services enable spanner.googleapis.com aiplatform.googleapis.com

## Set up Spanner
You will need a **Spanner** instance for the following stages of this notebook.

### Create a Spanner Instance & Database
Running the below cell will verify the existence of the Spanner instance & database and also create a new instance and database if one does not exist.

> ⏳ - Creating a Spanner instance & database ay take a few seconds.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud Spanner instance. Once filled in, run the cell.

# Please fill in these values.
region = "us-central1" #@param {type:"string"}
instance_id = "YOUR-INSTANCE-NAME" #@param {type:"string"}
database_id = "YOUR-DB-NAME" #@param {type:"string"}
# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_id, "⚠️ Please provide the name of your instance"
assert database_id, "⚠️ Please provide the name of your database"

# check if Cloud Spanner instance exists in the provided region
instance = !gcloud spanner instances describe {instance_id}

if 'ERROR' in instance[0]:
  print("Instance not found")
  print("Creating new Cloud Spanner instance...")
  !gcloud spanner instances create {instance_id} \
    --config={config} --nodes=4 --description={instance_id}

database = !gcloud spanner databases describe {database_id} --instance={instance_id}

if 'ERROR' in database[0]:
  print("Database not found")
  print("Creating new Cloud Spanner database...")
  !gcloud spanner databases create {database_id} \
    --instance={instance_id}

## Import data to your database

Now that you have your database, you will need to import data! We will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


To read data from a CSV file on Google Cloud Storage (GCS), installing `google-cloud-storage`.

In [ ]:
%pip install google-cloud-storage

You can read data from a csv in GCS and then insert it into Spanner table using batch transaction.

In [18]:
from google.cloud import storage
import csv
from io import BytesIO


def read_csv_from_gcs(bucket_name, file_name):
    # Initialize a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.get_bucket(bucket_name)

    # Get the blob (file)
    blob = bucket.blob(file_name)

    # Download the file
    content = blob.download_as_string()

    # Read CSV content
    csv_data = content.decode("utf-8")
    csv_reader = csv.reader(csv_data.splitlines())

    # Convert CSV reader to list of rows
    rows = list(csv_reader)

    return rows

In [ ]:
from google.cloud import spanner

client = spanner.Client()
instance = client.instance(instance_id)
database = instance.database(database_id)
table_name = "netflix_titles"

database.update_ddl(
    [
        """CREATE TABLE IF NOT EXISTS {} (
    show_id STRING(MAX) NOT NULL,
    type STRING(MAX),
    title STRING(MAX),
    director STRING(MAX),
    casta STRING(MAX),
    country STRING(MAX),
    date_added STRING(MAX),
    release_year INT64,
    rating STRING(MAX),
    duration STRING(MAX),
    listed_in STRING(MAX),
    description STRING(MAX)
) PRIMARY KEY (show_id)""".format(
            table_name
        )
    ]
)

In [20]:
rows = read_csv_from_gcs(
    "cloud-samples-data", "langchain/netflix_titles_compute_embeddings.csv"
)

batch_size = 500

for i in range(1, len(rows), batch_size):
    with database.batch() as batch:
        batch_rows = rows[i : i + batch_size]
        batch.insert(
            table=table_name,
            columns=[
                "show_id",
                "type",
                "title",
                "director",
                "casta",
                "country",
                "date_added",
                "release_year",
                "rating",
                "duration",
                "listed_in",
                "description",
            ],
            values=batch_rows,
        )

# **Use case 1: Spanner as a document loader**

Now that you have data in your database, you are ready to use Spanner as a document loader. This means we will pull data from the database and load it into memory as documents. We can then feed these documents into the vector store.

You can see we also pass in a query, table_name and a list of columns. The query tells the loader what query to use to pull data. The "content_columns" argument refers to the columns that will be used as "content" in the document object we will later construct. The rest of the columns in that table will become the "metadata" associated with the documents.

In [21]:
from langchain_google_spanner.loader import SpannerLoader

content_columns = ["title", "director", "casta", "description"]
loader = SpannerLoader(
    instance_id=instance_id,
    database_id=database_id,
    query=f"SELECT * FROM {table_name};",
    content_columns=content_columns,
)

Then let's run the function to pull our documents from out database using our document loader. You can see the first 5 documents from the database here. Nice, you just used  Spanner as a document loader!

In [22]:
documents = loader.load()
print(f"Loaded {len(documents)} from the database. 5 Examples:")
for doc in documents[:5]:
    print(doc)

Loaded 8802 from the database. 5 Examples:
page_content="The Starling Theodore Melfi Melissa McCarthy, Chris O'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, Kimberly Quinn, Loretta Devine, Ravi Kapoor A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward." metadata={'show_id': 's10', 'type': 'Movie', 'country': 'United States', 'date_added': 'September 24, 2021', 'release_year': 2021, 'rating': 'PG-13', 'duration': '104 min', 'listed_in': 'Comedies, Dramas'}
page_content='On the Verge Julie Delpy, Elisabeth Shue, Sarah Jones, Alexia Landeau, Mathieu Demy, Troy Garity, Timm Sharp, Giovanni Ribisi Four women — a chef, a single mom, an heiress and a job seeker — dig into love and work, with a generous side of midlife crises, in pre-pandemic LA.' metadata={'show_id': 's100', 'type': 'TV Show', 'country': 'France, United States', 'date_added': '

# **Use case 2: Spanner as Vector Store**

Now, let's learn how to put all of the documents we just loaded into a vector store so that we can use vector search to answer our user's questions!

### Create Your Vector Store table

Based on the documents that we loaded before, we want to create a table with a vector column as our vector store. We will start it by intializing a vector table by calling the `init_vectorstore_table` function from our `SpannerVectorStore`. As you can see we list all of the columns for our metadata.


In [ ]:
from langchain_google_spanner import SpannerVectorStore, TableColumn

sample_vector_table_name = "movie_vector_table_samples"


SpannerVectorStore.init_vector_store_table(
    instance_id=instance_id,
    database_id=database_id,
    table_name=sample_vector_table_name,
    metadata_columns=[
        TableColumn("show_id", "STRING(MAX)"),
        TableColumn("type", "STRING(MAX)"),
        TableColumn("country", "STRING(MAX)"),
        TableColumn("date_added", "STRING(MAX)"),
        TableColumn("release_year", "INT64"),
        TableColumn("rating", "STRING(MAX)"),
        TableColumn("duration", "STRING(MAX)"),
        TableColumn("listed_in", "STRING(MAX)"),
    ],
    primary_key="langchain_id",
)

### Try inserting the documents into the vector table

Now we will create a vector_store object backed by our vector table in the Spanner database. Let's load the data from the documents to the vector table. Note that for each row, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [26]:
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

# Initialize the embedding service. In this case we are using version 003 of Vertex AI's textembedding-gecko model. In general, it is good practice to specify the model version used.
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)


vector_store = SpannerVectorStore(
    embedding_service=embeddings_service,
    instance_id=instance_id,
    database_id=database_id,
    table_name=sample_vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "rating",
        "country",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

Now let's try to put the documents data into the vector table. Here is a code example to load the first 5 documents in the list.

In [ ]:
import uuid

docs_to_load = documents[:5]

# ! Uncomment the following line to load all 8,800+ documents to the database vector table with calling the embedding service.
# docs_to_load = documents

ids = [str(uuid.uuid4()) for i in range(len(docs_to_load))]
vector_store.add_documents(docs_to_load, ids, batch_size=500)

### Import the rest of your data into your vector table

You don't have to call the embedding service 8,800 times to load all the documents for the demo. Instead, we have prepared data with the all 8,800+ rows with pre-computed embeddings in a `.csv` file. Let's import data from csv directly.

In [28]:
vector_store_rows = read_csv_from_gcs(
    "cloud-samples-data", "langchain/spanner/netflix_titles_embeddings.csv"
)

for vector_store_row in vector_store_rows:
    # Cast embedding column from String to Array<Float>
    vector_store_row[2] = [float(x) for x in vector_store_row[2][1:-1].split(",")]

In [33]:
batch_size = 500

for i in range(0, len(vector_store_rows), batch_size):
    with database.batch() as batch:
        batch_rows = vector_store_rows[i : i + batch_size]
        batch.insert(
            table=sample_vector_table_name,
            columns=[
                "langchain_id",
                "content",
                "embedding",
                "show_id",
                "type",
                "country",
                "date_added",
                "release_year",
                "rating",
                "duration",
                "listed_in",
            ],
            values=batch_rows,
        )

# **Use case 3: Spanner as Chat Memory**

Next we will add chat history (called “memory” in the context of LangChain) to our application so the LLM can retain context and information across multiple interactions, leading to more coherent and sophisticated conversations or text generation. We can use Spanner as “memory” storage in our application so that the LLM can use context from prior conversations to better answer the user’s prompts. First let's initialize Spanner as memory storage.

In [34]:
from langchain_google_spanner import (
    SpannerChatMessageHistory,
)

message_table_name = "message_store"

SpannerChatMessageHistory.create_chat_history_table(
    instance_id=instance_id, database_id=database_id, table_name=message_table_name
)

chat_history = SpannerChatMessageHistory(
    instance_id=instance_id,
    database_id=database_id,
    table_name=message_table_name,
    session_id="my-test-session",
)

Here is an example of how you would add a user message and how you would add an ai message.

In [35]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

[HumanMessage(content='Hi!'),
 AIMessage(content="Hello there. I'm a model and am happy to help!")]

# **Conversational RAG Chain backed by Spanner**

So far we've tested with using Spanner as document loader, Vector Store and Chat Memory. Now let's use it in the `ConversationalRetrievalChain`.

We will build a chat bot that can answer movie related questions based on the vector search results

In [36]:
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_google_spanner import (
    SpannerChatMessageHistory,
    SpannerVectorStore,
    SpannerLoader,
)

# Intialize the embedding service
embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=project_id
)

# Intialize the Vector Store
vector_table_name = sample_vector_table_name
vector_store = SpannerVectorStore(
    embedding_service=embeddings_service,
    instance_id=instance_id,
    database_id=database_id,
    table_name=vector_table_name,
    metadata_columns=[
        "show_id",
        "type",
        "country",
        "rating",
        "date_added",
        "release_year",
        "duration",
        "listed_in",
    ],
)

# Intialize the SpannerChatMessageHistory
chat_history = SpannerChatMessageHistory(
    instance_id=instance_id,
    database_id=database_id,
    session_id="my-test-session",
    table_name=message_table_name,
)

Let's create a prompt for the LLM. Here we can add instructions specific to our application, such as "Don't make things up".

In [38]:
# Prepare some prompt templates for the ConversationalRetrievalChain
prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now let's use our vector store as a retreiver. Retreiver's in Langchain allow us to literally "retrieve" documents.

In [39]:
# Intialize retriever, llm and memory for the chain
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now let's intialize our LLM, in this case we are using Vertex AI's "gemini-pro".

In [40]:
llm = VertexAI(model_name="gemini-pro", project=project_id)

We clear our chat history, so that our application starts without any prior context to other conversations we have had with the application.

In [43]:
chat_history.clear()

memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

Now let's create a conversational retrieval chain. This will allow the LLM to use chat history in it's responses, meaning we can ask it follow up questions to our questions instead of having to start from scratch after each inquiry.

In [44]:
# create the ConversationalRetrievalChain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

# ask some questions
q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

# browser the chat history
chat_history.messages

Question: What movie was Brad Pitt in?
Answer: - Inglourious Basterds
- By the Sea
- Killing Them Softly
- Babel
- War Machine

Question: How about Jonny Depp?
Answer: - Charlie and the Chocolate Factory
- The Rum Diary
- What's Eating Gilbert Grape
- The Imaginarium of Doctor Parnassus

Question: Are there movies about animals?
Answer: Yes, there are movies about animals. From the vector search result, here are some examples: "Open Season," "If I Were an Animal," "The Animal People," "We're No Animals" and "Animals on the Loose: A You vs. Wild Movie."



[HumanMessage(content='What movie was Brad Pitt in?'),
 AIMessage(content='- Inglourious Basterds\n- By the Sea\n- Killing Them Softly\n- Babel\n- War Machine'),
 HumanMessage(content='How about Jonny Depp?'),
 AIMessage(content="- Charlie and the Chocolate Factory\n- The Rum Diary\n- What's Eating Gilbert Grape\n- The Imaginarium of Doctor Parnassus"),
 HumanMessage(content='Are there movies about animals?'),
 AIMessage(content='Yes, there are movies about animals. From the vector search result, here are some examples: "Open Season," "If I Were an Animal," "The Animal People," "We\'re No Animals" and "Animals on the Loose: A You vs. Wild Movie."')]